### Scenario 1

...................

.......2...2.......


...................

..........3...3....


..............4....

..................4


### Scenario 2

...................

........2..2.......


...........3.......

..............3....


...............4...

..................4



### Scenario 3

.....1.............

........2..2.......


.......2...........

...........3..3....


..........3........

...............4..4

### Scenario 4

...................

.........2.2.......


............3......

..............3....


................4..

..................4

### Scenario 5

...........5.......

.........2.2.......


................5..

..........1...3....


..5................

............2.....4


### Scenario 6

...5.......3.......

...5...100.1..2....


........5......4...

......300.1..2...3.


.5...........5.....

..4...00.1..2...3..

In [1]:
import random as rd
import scipy as sc
import matplotlib.pyplot as plt
%matplotlib inline

In [25]:
class TwoLaneTrafficSimulator:
    
    def __init__(self, road_len, p_car, v_max, p_slow, p_change = 1, verbose = True ):
        """
        Initialize traffic simulation:
        road_lend: length of the road (circumference)
        p_car: car density
        v_max: maximum speed
        p_slow: probability of slowing down 1 speed unit
        verbose: whether we want to display the states
        """      
        self.road_len = road_len
        self.v_max = v_max
        self.p_slow = p_slow
        self.verbose = verbose       
        self.p_change = p_change
        self.state1 = sc.zeros([road_len], int)
        self.state2 = sc.zeros([road_len], int)
        
        # All cars start at full speed
        for i in range(road_len):
            if rd.random() < p_car: self.state1[i] = self.v_max 
            else: self.state1[i] = -1    
            if rd.random() < p_car: self.state2[i] = self.v_max       
            else: self.state2[i] = -1
        
        
    def display(self):
        """
        Method for visualizing the state of the model
        """     
        print(''.join('.' if x == -1 else str(x) for x in self.state1))        
        print(''.join('.' if x == -1 else str(x) for x in self.state2))
        print

        
    def update(self):
        """
        Advances the simulation 1 timestep forward by doing the following on all cars:
         - Increase speed +1 if v smaller than v_max and distance to next vehicle > v+1
         - if distance to next car < v, slow down to d-1.
         - If speed > 0, randomly decrease speed by 1.
         - Display
         - Advance all cars v steps.
        """      
        flow = 0     
        
        
        copy_state1 = self.state1[:] 
        copy_state2 = self.state2[:]
        #check whether cars want to change
        for i, v in enumerate(self.state1):
            l = v+1
            d = 0
            #look front
            while self.state1[(i + d) % self.road_len] == -1: d += 1                            
            if l >= d:
                d_o = 1
                #look front other lane
                while self.state2[(i + d_o) % self.road_len] == -1: d_o += 1 
                if d_o - 1 > l:
                    #look back other lane
                    d_o = 1
                    while self.state2[(i - d_o) % self.road_len] == -1: d_o += 1 
                    if d_o - 1 > self.v_max:
                        if rd.random() < self.p_change:
                            copy_state1[i] = -1
                            copy_state2[i] = v

        for i, v in enumerate(self.state2):
            l = v+1
            d = 0
            #look front
            while self.state2[(i + d) % self.road_len] == -1: d += 1                            
            if l >= d:
                d_o = 1
                #look front other lane
                while self.state1[(i + d_o) % self.road_len] == -1: d_o += 1 
                if d_o - 1 > l:
                    #look back other lane
                    d_o = 1
                    while self.state1[(i - d_o) % self.road_len] == -1: d_o += 1 
                    if d_o - 1 > self.v_max:
                        if rd.random() < self.p_change:
                            copy_state2[i] = -1
                            copy_state1[i] = v
         
        self.state1 = copy_state1[:]
        self.state2 = copy_state2[:]
              
        for i, v in enumerate(self.state1):
            if v >= 0:
                #get distance to next car
                d = 1
                while self.state1[(i + d) % self.road_len] == -1:
                    d += 1
                #update speeds
                if d > v + 1:
                    self.state1[i] = min(self.state1[i] + 1, self.v_max)
                else:
                    self.state1[i] = d - 1
                if self.state1[i] > 0 and rd.random() < self.p_slow:
                    self.state1[i] -= 1
                #Count flow
                if i + v > self.road_len: flow += 1
                    
                    
        for i, v in enumerate(self.state2):
            if v >= 0:
                #get distance to next car
                d = 1
                while self.state2[(i + d) % self.road_len] == -1:
                    d += 1
                #update speeds
                if d > v + 1:
                    self.state2[i] = min(self.state2[i] + 1, self.v_max)
                else:
                    self.state2[i] = d - 1
                if self.state2[i] > 0 and rd.random() < self.p_slow:
                    self.state2[i] -= 1
                #Count flow
                if i + v > self.road_len: flow += 1        
        
        
                        
        #We display before moving the cars
        if self.verbose: self.display()    
                
        copy_state = self.state1[:]
        i = 0
        while i < len(copy_state):
            v = copy_state[i]
            if v >= 0:
                self.state1[i] = -1
                i += v
                self.state1[i % self.road_len] = v
                i += 1
            else: 
                i+=1 
        
        copy_state = self.state2[:]
        i = 0
        while i < len(copy_state):
            v = copy_state[i]
            if v >= 0:
                self.state2[i] = -1
                i += v
                self.state2[i % self.road_len] = v
                i += 1
            else: 
                i+=1 
        
        return flow/2.0
    
    
    
    def run(self, n_steps = 100):
        if self.verbose: self.display()  
        flow_sum = 0.0
        for step in range(n_steps):
            flow_sum += self.update()
        return flow_sum / n_steps

In [26]:
a = TrafficSimulator(100, .1, 5, .5)
a.run(10)

..55..........5..5.........55.........5.....55.......5.....5..5.............5.5....5...5.......5....
55.5.......55.............5....5...................................................5...........5....

..05..........2..5.........05...............04.............1..4.............0.4........5.......4....
01.4.......05.............4....4......5..............5.............................4...........4....

..0.....4.......2.....3....0.....5..........1....4..........2.....4.........0.....5.........4......2
0.1....2...1.....5............4....4.......5..............5............................5...........0

.01.........4.....3......0.1..........4......1.......5........2.......4.....0..........5........3...
1..2.....2..1.........4...........4....5........4..............5............................5......0

.0.2.......2....4....3...1..2.............2...2...........4.....2.........1.0...............4......1
.1...3.......1............4...........4.....4.......4...............5..................

0.5